# MY475 Seminar 1: computation graphs and backpropagation by hand

In this seminar, we are going to construct a simple perceptron model from scratch, following the computation graphs introduced in Lecture 2. We will:

* Define a Value class to build a computation graph
* Implement forward and backward passes
* Train our model using gradient descent

This tutorial is based on the amazing tutorial provided by Andrej Karpathy in his Neural Networks: Zero to Hero YouTube series. You can find the original video [here](https://www.youtube.com/watch?v=VMj-3S1tku0&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=1). The early part of today's exercises will mirror his implementation. We will diverge when implementing a model: I want to focus on re-implementing the simple perceptron model graphs we discussed in Lecture 2.

Looking forward, in Seminar 2, we will pivot to using **pytorch** -- an industry standard library for constructing (deep) neural networks. While this library will allow us to abstract away from some of the particulars we cover today, it is worth noting that pytorch models are very object-oriented and so the key skills learned in this seminar will be directly transferable to the next.

## Exercises

#### Defining the computation graph

1. Using the code chunk below, define a **Value** class that has a *data* attribute. Redefine the "magic method" `__repr__` to return a string that includes that data value of the object.
   
2. In the same code chunk, add new magic methods for addition and multiplication. You can call the second argument whatever you wish: one option would be to use `other` (i.e. self + other). These methods should return **Value** objects with the sum and product of the data, respectively.

*Great! We now have the very basics of a computation graph (at least, we have enough to go forwards!) Try it out. Construct a simple computation graph for the function $f(x) = 2x + 7$ -- you'll need to intialise $x$ with some value to see the calculation in action.*
   
3. In class, our value nodes also had a gradient attribute (let's call it `grad` here). Initialise the gradient of any new **Value** object to 0. 

4. When constructing computation graphs, recall that we calculate the gradient at any given Value node using the chain rule, and the known derivatives of the operations we perform. For any given operation on one (or two) Value nodes, when we backpropagate the gradient, we update the gradients of those nodes. Hence, whenever we create a new Value node as part of an operation, we need to define a function that allows us to backpropagate the gradient through that operation. We will call this method `_backward`. Amending your Value class:

   * Add a default `_backward` method that does nothing to the class
   * For both addition and multiplication magic methods, define a `_backward` function that updates the gradient of the input Values to that function. Set the `_backward` method of the output Value to this newly defined function (you want to set a method, not the result of calling that method).

5. For the next step, it will be helpful to keep a track of which Value nodes are used to calculate a given Value node. Add a `_children` attribute to the Value class with a default value of an empty tuple. Then in both operation magic methods, when defining the new Value, initialise it with the Value nodes used in the operation.

6. We are almost there! Each Value now records which other Value nodes (if any) determine its own data (the `_children`). But, there is no single map that charts how data flows through our graph as a whole. What we want to do is define a "topological" map of the graph. If you think about our visualisations of graphs, these are like trees where the output is our root, and the branches reflect the intermediary operations required to compute the final value, where the terminal (or leaf) nodes are the input data.

   * When working with trees, it is quite common to use recursive methods to build or map the graph as a whole. In the final code chunk of this exercise sheet, I have provided you the skeleton code to build this map, using recursion. The `___` components reflect items you need to complete. As you work through this algorithm, try and iterate through a simple graph to work out what gets added where, and when!

7. Now we can complete our implementation of backpropagation! Consider the final output node of the network. We start with the final node in the graph (which will have a gradient of 1), then iteratively move backwards through the connected nodes, propagating back the gradient. So, in case a Value node is the final one, we need to define a `backward()` method that a) builds the topological map of the graph, b) iterates through the nodes in reverse order, and c) backpropagates the gradient to that node's children:

   * Define a `backward` method for the Value class, pasting in the code you completed in the previous step
   * Build a map starting at the Value node (still inside the `backward` method)
   * Set the gradient of the current Value node
   * Use a for loop to iterate through the map in reverse order, calling the `_backward` method of each Value node 


#### Training a perceptron model (i.e. a single neuron)

From the lecture, let's suppose the ground truth function of a neuron with weight $w$ and bias term $b$ is $y = f(x) = 2x + 0$. Let's build the computation graph:

8. In the Data code chunk, define two Value objects, `x` and `y`, with values 1 and 2, respectively. Then define Value objects for `w` and `b` initialising both to 0. Finally, define the computation graph by creating nodes `a` (the product of `x` and `w`) and `y_pred` (the sum of `a` and `b`).

9. This is the bit that I find magic! Calculating the loss merely means appending further calculations to the end of the graph. Frist calculate the `err` as the difference between `y_pred` and the ground truth `y`. Then calculate the final `loss` as the square of the error.

   * Run this code chunk -- you should get an error! This is because we haven't overwritten the definitions of subtraction and raising a Value to a power. 
     
     * Add the __sub__ method to your Value class. One way to do this is to completely redefine the subtraction method, which would require implementing a `_backward` routine. A geekier way 🤓 that avoids this tedium is to redefine `__neg__(self)` to return `-1*self` then define __sub__ to be the *addition* of self and the negative of your second Value. 

     * To redefine the power operator, we use the `__power__` magic method. Here you will need to define the `_backward` routine. (Hint: you can use the derivative cheatsheet from Lecture 1 and assume that we will only raise a Value to an integer/float).
   
   * Run the code chunk again. It should now work!

10. Let's see backpropagation in action! Call the `backward()` method on the `loss` Value object. Then print the gradients of `w` and `b`.

11. Now we can implement gradient descent! In the next code chunk:
   
   *  Define a learning rate of 0.1, and update the values of `w` and `b` using the gradient descent formula
   *  Wrap this routine in a for loop that iterates 100 times
   *  Print the values of `w` and `b` after executing the loop

12. Finally, as in the lecture, let's introduce more data. Define a list `xs` that contains Values 1 and 3. Define a corresponding list `ys` of values that are twice the `x` values. Then, adapt your training loop to iterate over these two examples within *each* epoch, updating the weights and biases after each example. As before, your training loop should iterate 100 times. Inspect the new values of `w` and `b` after training.

### Value class

In [9]:
class Value:
    def __init__(self, data, _children=()):
        self.data = data
        self.grad = 0
        self._backward = lambda: None
        self._children = _children

    # output format
    def __repr__(self):
        return f'Value:{self.data}, Gradient: {self.grad}, Children: {self._children}'

    # operations
    def __add__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        out = Value(self.data + other.data, (self, other))

        def _backward():
            self.grad += out.grad * 1
            other.grad += out.grad * 1

        out._backward = _backward
        return out

    def __mul__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        out = Value(self.data * other.data, (self, other))

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward

        return out

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __pow__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        out = Value(self.data ** other.data, (self, other))

        def _backward():
            self.grad += other.data * (self.data ** (other.data - 1)) * out.grad
            other.grad += (self.data ** other.data) * out.grad

        out._backward = _backward

        return out

    def __truediv__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        out = Value(self.data / other.data, (self, other))

        def _backward():
            self.grad += (1 / other.data) * out.grad
            other.grad += (-self.data / (other.data ** 2)) * out.grad

        out._backward = _backward

        return out

    def abs (self):
        return Value(abs(self.data), (self,))


    # backpropagation
    def backward(self):
        visited = set()
        graph_map = []

        def build_graph(node):
            if node not in visited:
                visited.add(node)
                for child in node._children:
                    build_graph(child)
                graph_map.append(node)

        build_graph(self)

        self.grad = 1
        graph_map.reverse()
        for node in graph_map:
            node._backward()

        return graph_map

## Data, loss, and gradient descent

### Data, computation, and backpropagation of a single example

In [10]:
w = Value(0)
b = Value(0)

for i in range(1000):
    x = Value(1)
    y = Value(2)

    a = x * w
    y_pred = a + b

    err = y_pred - y
    loss = err ** 2

    w.grad = 0
    b.grad = 0

    loss.backward()
    w.grad, b.grad

    # update weights and bias using gradient descent formula
    lr = 0.01
    w.data -= lr * w.grad
    b.data -= lr * b.grad

print(w.data, b.data)

0.9999999999999987 0.9999999999999987


### Stochastic gradient descent

In [11]:
import random

w = Value(0)
b = Value(0)

for i in range(1000):

    xs = random.randint(1,3)
    x = Value(xs)
    y = Value(2*xs)

    a = x * w
    y_pred = a + b

    err = y_pred - y
    loss = err ** 2

    w.grad = 0
    b.grad = 0

    loss.backward()
    w.grad, b.grad

    # update weights and bias using gradient descent formula
    lr = 0.01
    w.data -= lr * w.grad
    b.data -= lr * b.grad

print(w.data, b.data)

1.9687639113930773 0.07072716507369836


## Extension 1: altering the activation function

In Lecture 2, although we used the identity function, we could have used any other activation function. Try implement a [ReLU activation function](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)). You will need to know the derivative of the ReLU function to complete this exercise, in order to implement the corresponding `_backward` method. You will also need to amend the computation such that this activation is called prior to calculating the error.

In [12]:
import random

w = Value(0)
b = Value(0)

for i in range(1000):

    xs = random.randint(1,3)
    x = Value(xs)
    y = Value(2*xs)

    a = x * w
    y_pred = a + b
    pos = y_pred + y_pred.abs()
    relu = pos / 2

    err = relu - y
    loss = err ** 2

    w.grad = 0
    b.grad = 0

    loss.backward()
    w.grad, b.grad

    # update weights and bias using gradient descent formula
    lr = 0.001
    w.data -= lr * w.grad
    b.data -= lr * b.grad

print(w.data, b.data)

1.7062930258900773 0.6506436947887849


## Extension 2: building a multi-layer perceptron

We won't have time to cover multilayer perceptions in this seminar, and dong so depends on some of the terminology we will only cover in depth next week. For those who are interested, and want to extend their own implementation of neural networks, I would strongly recommend following on with Karpathy's tutorial. You can pick up his video at around 1:44:00 to see an implementation of a multilayer perceptron. 

There are two important differences between our implementation and Karpathy's if you try and continue with his implementation:

1. We assume identity activation functions, whereas Karpathy uses a tanh activation function. This requires adding a new `tanh()` method to the Value class, as well as the corresponding backwards routine.
2. We assume that all data is of Value class, but sometimes it might be helpful to be able to handle ints and floats directly. In the various operation magic methods, you can add a check to convert any second argument to a Value object if it is not already.
